# Refresh questions

- @author Gong Zequn (zequn.gong@u.nus.edu)
- @create 2021.07.17
- @modify 2021.07.22

We cannot just delete the old questions and then add new questions, since 
we are using question_id in the answer table to indicate which question 
does this answer belong to.

The SQL should be like:

```sql
BEGIN;
UPDATE `question` SET `enable`=0;
INSERT INTO `question` (`word`) VALUES ("a"), ("here") ON DUPLICATE KEY UPDATE `enable`=1;
COMMIT;
```

In [1]:
!pip install redis pymysql pandas

## Configs

In [2]:
# Common configs

question_list_file_name = "./input/list.txt"

class Configs:
    _db_host = 'mysql'
    _db_port = 3306
    _db_user = 'csqsiew'
    _db_passwd = 'u98x7v89asx'
    _db_name = 'singlishwords'
    _db_charset = 'utf8'

    _redis_host = 'redis'
    _redis_port = 6379

## Read questions

In [3]:
with open(question_list_file_name) as f:
    questions = f.read().splitlines()

## Init database connection

In [4]:
import libs.database as database

database.initConnections(Configs)

## Refresh questions in database

In [5]:
database.refreshMySQLQuestions(questions)

## Refresh the cache

In [6]:
database.refreshRedis()

## Check if success

This method check the current questions in the sample set.

In [7]:
database.redisConnection.llen("questionList")

4462